In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
from python_scripts.utils.classification_evals import *
from sklearn.linear_model import LogisticRegression, Perceptron, PassiveAggressiveClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
import warnings
# warnings.filterwarnings("ignore")

# Classification

In [41]:
final_dataset_path = '../../../dataset/final/windows/separate/'
acc_dataset_path = final_dataset_path + 'PatchTable_Acceleration_Filtered.csv'
gyro_dataset_path = final_dataset_path + 'PatchTable_Gyroscope_Filtered.csv'

In [42]:
acc_dataset_df = pd.read_csv(acc_dataset_path)
gyro_dataset_df = pd.read_csv(gyro_dataset_path)
acc_id_groupings = acc_dataset_df['Subject']
gyro_id_groupings = gyro_dataset_df['Subject']

In [43]:
acc_dataset_df

,Subject,Class,CorssCorrelationSum_VTAP,CorssCorrelationPeak_VTAP,IndexHarmonicity_ML,FirstDerivativeMean_SigComplete_VT,FirstDerivative_Moment3rd_SigComplete_VT,FirstDerivativeLogRatio_SigComplete_ML,FirstDerivativeMean_SigComplete_AP,FirstDerivative_Skweness_SigComplete_AP,...,Mean_SigComplete_Magnitude,Skewness_SigComplete_Magnitude,Percentile25_SigComplete_Magnitude,MaxFreqSpectrum_Below5Hz_Magnitude,RelativePower_5To10Hz_Magnitude,RelativePower_BelowDomFr_Magnitude,MedianPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude
0,1,0,-0.163081,-0.662803,0.528986,0.475757,-0.062359,-0.136782,-0.409614,-1.334350,...,-0.035799,1.275514,1.444325,-0.116140,1.224695,-0.156583,-0.165272,-0.146060,-0.086591,-1.771846
1,1,0,-0.163483,-0.668085,-0.364654,0.090667,-0.110018,-0.233595,-0.054282,-1.066276,...,-0.281341,-0.695417,1.432455,0.408305,1.251751,-0.063859,-0.224007,-0.146061,0.952947,-2.360037
2,1,0,-0.163548,-0.668120,-0.362345,-0.033453,-0.110054,-0.112613,-0.050146,-1.160794,...,-0.404884,-0.490985,1.412446,-2.039106,1.249730,0.167928,-0.224137,-0.146061,1.178809,-2.654132
3,1,0,-0.163525,-0.668035,-0.376434,0.009309,-0.110064,-0.227149,-0.042921,-1.174912,...,-0.463610,-0.346706,1.393891,-2.039106,1.052092,0.285549,-0.224224,-0.146061,1.203253,-2.801180
4,1,0,-0.163469,-0.667327,13.995720,-0.002560,-0.109835,-0.698540,0.162228,-0.055198,...,-0.208414,2.387815,1.407141,-1.951699,-1.596697,-0.016023,-0.196872,-0.146061,0.069855,-3.095275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,6,1,-0.152225,-0.607692,-0.416798,-0.867979,-0.047874,0.015603,-0.046169,0.002909,...,0.185887,0.225318,0.462994,-0.465771,0.163099,-0.191222,-0.209324,-0.146017,-0.295417,0.580916
1876,6,1,-0.150379,-0.623584,-0.154285,-0.524313,-0.062139,-0.028567,0.040108,0.172652,...,0.169964,0.472963,0.627831,-0.553178,0.996511,-0.178239,-0.219464,-0.146033,0.136042,-0.007274
1877,6,1,-0.148807,-0.580757,-0.314912,0.828930,-0.047780,-0.325827,-0.340791,0.078664,...,0.280245,0.255260,0.389285,1.194972,0.894499,-0.184609,-0.166815,-0.145960,-0.002798,0.139773
1878,6,1,-0.132307,-0.612842,-0.367696,1.526718,-0.039679,-0.599088,-2.453837,-3.100024,...,1.338794,0.849419,0.387508,1.194972,0.132428,-0.083886,0.621261,-0.145660,-0.065654,-0.007274


In [44]:
gyro_dataset_df

,Subject,Class,RelativePower_Below5Hz_Magnitude
0,1,0,-2.247382
1,1,0,-2.401791
2,1,0,-0.511308
3,1,0,-0.713828
4,1,0,0.242117
...,...,...,...
2054,6,1,0.432625
2055,6,1,0.203724
2056,6,1,1.628756
2057,6,1,1.868931


In [45]:
acc_target = acc_dataset_df.iloc[:, 1].copy().values
gyro_target = gyro_dataset_df.iloc[:, 1].copy().values
acc_features = acc_dataset_df.iloc[:, 2:].copy().values
gyro_features = gyro_dataset_df.iloc[:, 2:].copy().values

In [46]:
def get_model_scores(models_dict, features, target, id_groupings, groupk_folds):
    model_score_columns = list(scoring_methods.keys())
    model_score_columns.insert(0, 'Model')
    all_model_scores_df = pd.DataFrame(columns=model_score_columns)
    for model_key in models_dict:
        model_scores = cross_val_model(model=models_dict[model_key], features=features, target=target, id_groupings=id_groupings, groupk_folds=groupk_folds)
        model_scores['Model'] = model_key
        all_model_scores_df = pd.concat([all_model_scores_df, pd.DataFrame.from_dict(model_scores, orient='index').T], ignore_index=True)
    return all_model_scores_df

## Testing All Models

In [47]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Support Vector Machine': SVC(kernel='sigmoid'),
    'Gaussian Naive Bayes': GaussianNB(),
    'Bernoulli Naive Bayes': BernoulliNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'Voting': VotingClassifier(estimators=[('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()), ('knn', KNeighborsClassifier())]),
    'k-Nearest Neighbors': KNeighborsClassifier(n_neighbors=11),
    'Multi-layer Perceptron': MLPClassifier(max_iter=1000, hidden_layer_sizes=(2, 3, 2)),
    'Gaussian Process': GaussianProcessClassifier(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
    'Perceptron': Perceptron(),
    'Passive Aggressive': PassiveAggressiveClassifier()
}

In [48]:
all_model_scores_acc = get_model_scores(models, features=acc_features, target=acc_target, id_groupings=acc_id_groupings, groupk_folds=6)

In [49]:
all_model_scores_gyro = get_model_scores(models, features=gyro_features, target=gyro_target, id_groupings=gyro_id_groupings, groupk_folds=6)

In [50]:
all_model_scores_acc

,Model,Accuracy,Precision,Sensitivity,Specificity,F1
0,Logistic Regression,0.624847,0.618194,0.619095,0.540793,0.61009
1,Support Vector Machine,0.633188,0.616306,0.610036,0.484156,0.599424
2,Gaussian Naive Bayes,0.579185,0.605667,0.58201,0.750061,0.544592
3,Bernoulli Naive Bayes,0.586117,0.587931,0.586183,0.496324,0.565818
4,Decision Tree,0.556673,0.562103,0.546859,0.454773,0.53134
5,Random Forest,0.585936,0.562612,0.56184,0.393457,0.554058
6,Gradient Boosting,0.571411,0.560291,0.562882,0.438241,0.552581
7,AdaBoost,0.573327,0.56838,0.572235,0.519577,0.562507
8,Extra Trees,0.587876,0.577585,0.577574,0.399997,0.560825
9,Voting,0.591931,0.59896,0.587238,0.473567,0.578895


In [51]:
all_model_scores_gyro

,Model,Accuracy,Precision,Sensitivity,Specificity,F1
0,Logistic Regression,0.600973,0.596594,0.556517,0.243546,0.516311
1,Support Vector Machine,0.491224,0.480892,0.480854,0.383172,0.46968
2,Gaussian Naive Bayes,0.60224,0.598914,0.556426,0.240303,0.517205
3,Bernoulli Naive Bayes,0.557671,0.333354,0.52052,0.130237,0.388592
4,Decision Tree,0.538741,0.523832,0.524615,0.431121,0.517526
5,Random Forest,0.538741,0.522769,0.524088,0.431121,0.517526
6,Gradient Boosting,0.585256,0.57093,0.538107,0.21267,0.492399
7,AdaBoost,0.57861,0.579416,0.540821,0.229367,0.487922
8,Extra Trees,0.536254,0.519262,0.519194,0.423214,0.51926
9,Voting,0.570168,0.545178,0.539206,0.320312,0.519353
